In [5]:
import praw
import pandas as pd
import datetime as dt
from praw.models import MoreComments
import math
import json
import requests
import itertools
import numpy as np
import time
from datetime import datetime, timedelta

reddit = praw.Reddit(client_id='qGBDWGg0ooefOQ', \
                     client_secret='BXsX_M1rXMJVbRs6ED7fSJG77Bg', \
                     user_agent='Reddit Web Scraping', \
                     username='appliedintelligence', \
                     password='accenture2019')

In [6]:

def make_request(uri, max_retries = 5):
    def fire_away(uri):
        response = requests.get(uri)
        assert response.status_code == 200
        return json.loads(response.content)
    current_tries = 1
    while current_tries < max_retries:
        try:
            time.sleep(1)
            response = fire_away(uri)
            return response
        except:
            time.sleep(1)
            current_tries += 1
    return fire_away(uri)



def pull_posts_for(subreddit, start_at, end_at):
    
    def map_posts(posts):
        return list(map(lambda post: {
            'id': post['id'],
            'created_utc': post['created_utc'],
            'prefix': 't4_',
            'subreddit': subreddit
        }, posts))
    
    SIZE = 500
    URI_TEMPLATE = r'https://api.pushshift.io/reddit/search/submission?subreddit={}&after={}&before={}&size={}'
    
    post_collections = map_posts( \
        make_request( \
            URI_TEMPLATE.format( \
                subreddit, start_at, end_at, SIZE))['data'])
    n = len(post_collections)
    while n == SIZE:
        last = post_collections[-1]
        new_start_at = last['created_utc'] - (10)
        
        more_posts = map_posts( \
            make_request( \
                URI_TEMPLATE.format( \
                    subreddit, new_start_at, end_at, SIZE))['data'])
        
        n = len(more_posts)
        post_collections.extend(more_posts)
    return post_collections



def give_me_intervals(start_at,end_at, number_of_days_per_interval):
        
    ## 1 day = 86400,
    period = (86400 * number_of_days_per_interval)
    end = start_at + period
    yield (int(start_at), int(end))
    padding = 1
    while end <= end_at:
        start_at = end + padding
        end = (start_at - padding) + period
        yield int(start_at), int(end)


def pull_subreddit(subreddit_name, start_at, end_at):
    subreddit = subreddit_name
    posts = []
    for interval in give_me_intervals(start_at, end_at,7):
        pulled_posts = pull_posts_for(
            subreddit, interval[0], interval[1])
        
        posts.extend(pulled_posts)
        time.sleep(.500)
    return posts

subreddit_names = ['ttc', 'toronto','ontario']
start_at = math.floor(\
        (datetime.utcnow() - timedelta(days=365)).timestamp())
end_at = math.ceil(datetime.utcnow().timestamp())


In [4]:

topics_dict = { "title":[], 
                "score":[], 
                "id":[], 
                "url":[], 
                "comms_num": [], 
                "created": [], 
                "body":[]}


search_terms = ['ttc','presto', 'station','transit','subway', 'metrolinx','metro linx' ]
TIMEOUT_AFTER_COMMENT_IN_SECS = .350


final_posts = np.unique([post['id'] for post in posts ])

i = 1
for submission_id in final_posts:
  
    submission = reddit.submission(id=submission_id)
  #  posts_from_reddit.append(submission)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if any(word in comment.body.lower() for word in search_terms):
         #   comments_from_reddit.append(comment)
            topics_dict["title"].append(submission.title)
            topics_dict["score"].append(submission.score)
            topics_dict["id"].append(submission.id)
            topics_dict["url"].append(submission.url)
            topics_dict["comms_num"].append(submission.num_comments)
            topics_dict["created"].append(submission.created)
            topics_dict["body"].append(comment.body)
        
            if TIMEOUT_AFTER_COMMENT_IN_SECS > 0:
                time.sleep(TIMEOUT_AFTER_COMMENT_IN_SECS)
    print (i)
    i += 1


NameError: name 'posts' is not defined

In [ ]:
final_df = pd.DataFrame(topics_dict)
# final_df.to_csv('reddit_data_raw.csv')